# Illustration of mix-SQP solver applied to a small data set, and a large one

## Analysis setup

*Before attempting to run this Julia code, make sure your computer is properly set up to run this code by following the setup instructions in the README of the [git repository](https://github.com/stephenslab/mixsqp-paper).*

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
using RCall
using Random
using Printf
using LinearAlgebra
using SparseArrays
include("datasim.jl");
include("likelihood.jl");
include("mixEM.jl");
include("mixsqp_julia_v1.jl");
include("REBayes.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
Random.seed!(0);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [3]:
z = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(z,nv = 20);
L  = normlikmatrix(z,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm once to precompile the function.

In [5]:
out = mixsqp(L,verbose = false);

Observe that only a small number of iterations is needed to converge to the solution of the constrained optimization problem.

In [6]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixsqp(L,x = x0);

Running SQP algorithm with the following settings:
- 50000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 5.98e-07
iter      objective -min(g+1)  #nz #qp #ls
   1 2.62702453e+04 +4.98e-01   20
   2 1.92707133e+04 +9.47e+02    2  32   1
   3 1.23597568e+04 +3.35e+02    2  63   1
   4 1.10588873e+04 +1.47e+02    2  31   1
   5 1.07833333e+04 +6.98e+01    3  27   1
   6 1.05447236e+04 +3.37e+01    3  19   1
   7 1.03970757e+04 +1.69e+01    3  10   1
   8 1.01967794e+04 +8.31e+00    3  15   1
   9 1.00767996e+04 +4.23e+00    3   4   1
  10 9.98533249e+03 +2.02e+00    3   9   1
  11 9.87534018e+03 +9.37e-01    3  16   1
  12 9.83686386e+03 +3.93e-01    3   7   1
  13 9.81375860e+03 +1.18e-01    4   9   1
  14 9.80470988e+03 +2.87e-02    4   4   1
  15 9.80158446e+03 +1.32e-03    4   2   1
  16 9.80122388e+03 -3.17e-07    4   2   1
Optimization took 16 iterations and 0.3092 seconds.


## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [7]:
Random.seed!(1);
z = normtmixdatasim(round(Int,1e5));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 40$ normal densities.

In [8]:
sd = autoselectmixsd(z,nv = 40);
L  = normlikmatrix(z,sd = sd);
size(L)

(100000, 40)

## Fit mixture model using SQP algorithm 

Even on this much larger data set, only a small number of iterations is needed to compute the solution.

In [9]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixsqp(L,x = x0);

Running SQP algorithm with the following settings:
- 100000 x 40 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 1.35e-06
iter      objective -min(g+1)  #nz #qp #ls
   1 6.21692352e+04 +6.60e-01   40
   2 4.28411526e+04 +1.06e+03    2  55   1
   3 2.56096309e+04 +3.69e+02    3 100   1
   4 2.18424879e+04 +1.61e+02    2  52   1
   5 2.12479175e+04 +7.52e+01    2  40   1
   6 2.08873601e+04 +3.64e+01    2  37   1
   7 2.05703467e+04 +1.80e+01    3  29   1
   8 2.02435366e+04 +8.95e+00    3  30   1
   9 1.99723675e+04 +4.38e+00    3  25   1
  10 1.97676896e+04 +2.08e+00    3  15   1
  11 1.96306393e+04 +9.38e-01    3  12   1
  12 1.95365433e+04 +4.04e-01    4  22   1
  13 1.94905417e+04 +1.44e-01    5  31   1
  14 1.94697537e+04 +3.98e-02    5  39   1
  15 1.94618061e+04 +2.97e-03    4  13   1
  16 1.94607553e+04 -3.12e-07    4  15   1
Optimization took 16 iterations and 0.4986 seconds.


With no low-rank approximation (`lowrank = "none"`), the algorithm still converges even when using a very small correction factor `eps = 1e-12`.

In [10]:
out = mixsqp(L,x = x0,lowrank = "none",eps = 1e-12);

Running SQP algorithm with the following settings:
- 100000 x 40 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- Exact derivative computation (partial QR not used).
iter      objective -min(g+1)  #nz #qp #ls
   1 6.21694226e+04 +6.60e-01   40
   2 4.35976361e+04 +2.74e+08    2  52   1
   3 2.63754248e+04 +9.42e+07    3  53   1
   4 2.26716550e+04 +4.11e+07    3  33   1
   5 2.22369707e+04 +1.93e+07    3  18   1
   6 2.20822494e+04 +9.86e+06    3  10   1
   7 2.17850757e+04 +4.96e+06    3  43   1
   8 2.15686850e+04 +2.48e+06    2  11   1
   9 2.13181590e+04 +1.28e+06    3  39   1
  10 2.11397369e+04 +6.40e+05    2  11   1
  11 2.08812795e+04 +3.39e+05    3  39   1
  12 2.07941249e+04 +1.75e+05    3  10   1
  13 2.04915356e+04 +8.97e+04    3  37   1
  14 2.03990501e+04 +4.57e+04    3  10   1
  15 2.01797507e+04 +2.27e+04    3  16   1
  16 2.00663424e+04 +1.17e+04    3  10   1
  17 2.00068966e+04 +5.91e+03    3  10   1
  18 1.98269277e+04 +3.06e+03    

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [11]:
using Pkg
Pkg.status();
versioninfo();

    Status `~/.julia/environments/v1.0/Project.toml`
  [7d9fca2a] Arpack v0.3.0
  [336ed68f] CSV v0.4.3
  [159f3aea] Cairo v0.5.6
  [a81c6b42] Compose v0.7.1
  [f65535da] Convex v0.11.3
  [a93c6f00] DataFrames v0.14.1
  [31c24e10] Distributions v0.16.4
  [5789e2e9] FileIO v1.0.6
  [186bb1d3] Fontconfig v0.2.0
  [f6369f11] ForwardDiff v0.10.3
  [c91e804a] Gadfly v1.0.0
  [2e9cd046] Gurobi v0.5.5
  [7073ff75] IJulia v1.14.1
  [916415d5] Images v0.18.0
  [b6b21f68] Ipopt v0.4.3
  [4076af6c] JuMP v0.18.4
  [67920dd8] KNITRO v0.5.2
  [093fc24a] LightGraphs v1.2.0
  [898213cb] LowRankApprox v0.2.1
  [e1d29d7a] Missings v0.4.0
  [6405355b] Mosek v0.9.11
  [9b87118b] PackageCompiler v0.5.1
  [2dcacdae] ParallelDataTransfer v0.5.0
  [91a5bcdd] Plots v0.21.0
  [d330b81b] PyPlot v2.6.3
  [dca85d43] QuartzImageIO v0.6.0
  [6f49c342] RCall v0.13.0
  [ce6b1742] RDatasets v0.6.1
  [f2b01f46] Roots v0.7.3
  [276daf66] SpecialFunctions v0.7.2
  [2913bbd2] StatsBase v0.26.0
  [4c63d2b9] StatsFuns v0.7.1